In [1]:
from clear import CLEAR1
from rslvq import RSLVQ
import pandas as pd
import numpy as np
from density_check import Kernel_obj

In [2]:
b_c_data = pd.read_csv('data.csv')
b_c_data.columns
b_c_data.drop(['id', 'Unnamed: 32'], axis = 1, inplace= True)
label = b_c_data['diagnosis'].replace({'M': 0, 'B': 1})
train_data = b_c_data.drop(['diagnosis'], axis = 1)
train_data

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [3]:
from sklearn.preprocessing import MinMaxScaler
scale = MinMaxScaler()
X = scale.fit_transform(train_data)
norm_t_data = pd.DataFrame(X, columns = train_data.columns, index = train_data.index)

In [4]:
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
x_train, x_val, y_train, y_val = train_test_split(np.array(norm_t_data), np.array(label), test_size=0.3, random_state=42)

In [5]:
trained_model = RSLVQ(num_prototypes_per_class=1)

In [6]:
prototypes,_ = trained_model.fit(x_train, y_train)

Training: 100%|██████████| 100/100 [00:02<00:00, 39.75epoch/s]


In [7]:
trained_model.evaluate(x_val, y_val)

98.24561403508771

In [8]:
lab = pd.DataFrame(np.array(label), columns = ['labels'])

In [9]:
malignant_cases = norm_t_data.loc[lab['labels'] == 0]

In [10]:
lab

,labels
0,0
1,0
2,0
3,0
4,0
...,...
564,0
565,0
566,0
567,0


In [11]:
kern = Kernel_obj(np.array(norm_t_data.loc[lab['labels'] == 1]))

In [12]:
kern.kde_density(norm_t_data.iloc[0])

-0.0033955065488356366

In [13]:
p = []
for i in range(len(norm_t_data)):
    p.append(kern.kde_density(norm_t_data.iloc[i]))

In [14]:
np.array(p).max()

1.0

In [15]:
new_explainer = CLEAR1(trained_model, 100,synthetic_generator_scheme = 'Point-Specific', density_estimator= 'KDE', classification_threshold = 0.45, number_of_CFEs = 200, wachter_search_max = 300, cat_cols=['area_mean'], set_immutable=['radius_mean', 'texture_mean'])

In [16]:
new_explainer.fit(norm_t_data, lab)

In [60]:
new_explainer = CLEAR1(trained_model, 1000, neighbourhood= 'balanced', 
                       synthetic_generator_scheme = 'Perturbation', density_estimator = 'KDE',  classification_threshold = 0.5, 
                       density_threshold= 0.4, number_of_CFEs = 200, wachter_search_max = 300)
new_explainer.fit(norm_t_data, lab)
cf, es_cf= new_explainer.generate_counterfactual(norm_t_data.iloc[3])

C:\Users\amade\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [61]:
kern.kde_density(cf)

0.6617251854955143

In [19]:
np.array(cf - norm_t_data.iloc[3])

array([[ 0.02661259, -0.06452438,  0.00264959,  0.02517703, -0.34950892,
        -0.48384806, -0.35423246, -0.3057457 , -0.33426304, -0.5635568 ,
        -0.05928132,  0.01101206, -0.05268425, -0.00881054, -0.05035958,
        -0.30889973, -0.05983162, -0.12919813, -0.42522707, -0.14898648,
        -0.04213115, -0.06157262, -0.04573068, -0.00167402, -0.43361831,
        -0.51333314, -0.3218608 , -0.48686265, -0.60042095, -0.47494243]])

In [20]:
lab

,labels
0,0
1,0
2,0
3,0
4,0
...,...
564,0
565,0
566,0
567,0


In [21]:
cf

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
879,0.236703,0.296314,0.236151,0.128083,0.461812,0.327513,0.211372,0.217117,0.442,0.436443,...,0.206179,0.324355,0.195616,0.092334,0.481854,0.300679,0.226781,0.398017,0.399579,0.298769


In [22]:
cf-norm_t_data.iloc[2]

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
879,-0.364793,-0.093946,-0.359592,-0.321334,-0.052497,-0.103503,-0.25114,-0.418569,-0.067596,0.225196,...,-0.350207,-0.03572,-0.312826,-0.282174,-0.001736,-0.084697,-0.132963,-0.437034,-0.004127,0.085336


In [23]:
norm_t_data[lab['labels'] == 1].mode().iloc[0]['radius_mean']

0.15518954990770978

In [24]:
len(norm_t_data.iloc[2].copy())

30

In [25]:
cf

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
879,0.236703,0.296314,0.236151,0.128083,0.461812,0.327513,0.211372,0.217117,0.442,0.436443,...,0.206179,0.324355,0.195616,0.092334,0.481854,0.300679,0.226781,0.398017,0.399579,0.298769


In [26]:
norm_t_data.loc[lab['labels'] == 1].mean(axis = 0)

radius_mean                0.244476
texture_mean               0.277469
perimeter_mean             0.236925
area_mean                  0.135436
smoothness_mean            0.359733
compactness_mean           0.186199
concavity_mean             0.107914
concave points_mean        0.127820
symmetry_mean              0.344374
fractal_dimension_mean     0.271849
radius_se                  0.062496
texture_se                 0.190103
perimeter_se               0.058584
area_se                    0.026771
smoothness_se              0.186385
compactness_se             0.144097
concavity_se               0.065648
concave points_se          0.186733
symmetry_se                0.178728
fractal_dimension_se       0.094705
radius_worst               0.193874
texture_worst              0.306372
perimeter_worst            0.182260
area_worst                 0.091845
smoothness_worst           0.355210
compactness_worst          0.150753
concavity_worst            0.132778
concave points_worst       0

In [27]:
norm_t_data.iloc[2]

radius_mean                0.601496
texture_mean               0.390260
perimeter_mean             0.595743
area_mean                  0.449417
smoothness_mean            0.514309
compactness_mean           0.431017
concavity_mean             0.462512
concave points_mean        0.635686
symmetry_mean              0.509596
fractal_dimension_mean     0.211247
radius_se                  0.229622
texture_se                 0.094303
perimeter_se               0.180370
area_se                    0.162922
smoothness_se              0.150831
compactness_se             0.283955
concavity_se               0.096768
concave points_se          0.389847
symmetry_se                0.205690
fractal_dimension_se       0.127006
radius_worst               0.556386
texture_worst              0.360075
perimeter_worst            0.508442
area_worst                 0.374508
smoothness_worst           0.483590
compactness_worst          0.385375
concavity_worst            0.359744
concave points_worst       0

In [28]:
cf


,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
879,0.236703,0.296314,0.236151,0.128083,0.461812,0.327513,0.211372,0.217117,0.442,0.436443,...,0.206179,0.324355,0.195616,0.092334,0.481854,0.300679,0.226781,0.398017,0.399579,0.298769


In [29]:
es_cf

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,-142.149034,-6.730307,-150.779048,-8315.515287,-7.063436,16.219015,-1.940906,-1.517383,175.932523,4.841332,...,-11.676076,-5.418281,-7.559553,-190.510342,-2.167585,-6.280738,-2.235629,-1.243863,-1.915985,-9.564705


In [30]:
def l0_norm(vector):
    """
    Calculate the L0 norm of a vector, which is the count of non-zero elements in the vector.

    Args:
    - vector: The input vector (list or array).

    Returns:
    - The L0 norm of the vector (count of non-zero elements).
    """
    count = 0
    for element in vector:
        if element != 0:
            count += 1
    return count

In [31]:
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(norm_t_data, lab, test_size=0.3, random_state=42)  # 70% training and 30% test

In [32]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(hidden_layer_sizes=(6,5),
                    random_state=42,
                    verbose=True,
                    learning_rate_init=0.05)

# Fit data onto the model
clf.fit(X_train,y_train)

Iteration 1, loss = 0.64077087
Iteration 2, loss = 0.53083030
Iteration 3, loss = 0.42406778
Iteration 4, loss = 0.35559745
Iteration 5, loss = 0.31591907
Iteration 6, loss = 0.28053929
Iteration 7, loss = 0.25997796
Iteration 8, loss = 0.24583135
Iteration 9, loss = 0.23095861
Iteration 10, loss = 0.21229214
Iteration 11, loss = 0.18901531
Iteration 12, loss = 0.17805540
Iteration 13, loss = 0.16492963
Iteration 14, loss = 0.15340346
Iteration 15, loss = 0.14085805
Iteration 16, loss = 0.12920563
Iteration 17, loss = 0.12152019
Iteration 18, loss = 0.11406115
Iteration 19, loss = 0.10554150
Iteration 20, loss = 0.10055059
Iteration 21, loss = 0.09590339
Iteration 22, loss = 0.09356881
Iteration 23, loss = 0.08469624
Iteration 24, loss = 0.08436484
Iteration 25, loss = 0.08340997
Iteration 26, loss = 0.07402309
Iteration 27, loss = 0.07909781
Iteration 28, loss = 0.07107394
Iteration 29, loss = 0.07082861
Iteration 30, loss = 0.06571904
Iteration 31, loss = 0.06613034
Iteration 32, los

C:\Users\amade\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1118: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPClassifier(hidden_layer_sizes=(6, 5), learning_rate_init=0.05,
              random_state=42, verbose=True)

In [33]:
clf.predict_proba(norm_t_data)

array([[1.00000000e+00, 3.13843157e-18],
       [1.00000000e+00, 9.09092192e-11],
       [1.00000000e+00, 4.34196525e-16],
       ...,
       [9.99993995e-01, 6.00511964e-06],
       [1.00000000e+00, 3.84944956e-22],
       [1.06250416e-02, 9.89374958e-01]])

In [34]:
new_explainer = CLEAR1(clf, 1000,backend = 'sklearn', synthetic_generator_scheme='Point-Specific',cat_cols = [ 'area_mean', 'perimeter_mean'],  classification_threshold = 0.90, number_of_CFEs = 200)
new_explainer.fit(norm_t_data, lab)
new_explainer.synthetic_generator(norm_t_data, lab, norm_t_data.iloc[2], 0)
new_explainer.Balanced_Neighbourhood(norm_t_data.iloc[2], 1)

no numerical column specified
no numerical column specified


(     radius_mean  texture_mean  perimeter_mean  area_mean  smoothness_mean  \
 0       0.835437      0.390260        0.595743   0.449417         0.514309   
 1       1.143790      0.390260        0.595743   0.449417         0.514309   
 2       0.835437      0.867885        0.595743   0.449417         0.514309   
 3       0.835437      0.867885        0.110290   0.449417         0.514309   
 4       0.835437      0.867885        0.110290   0.220233         0.514309   
 ..           ...           ...             ...        ...              ...   
 994     2.166255      2.366824        0.281667   0.175864         3.043830   
 995     2.471581      2.174768        0.519729   0.176331         2.506755   
 996     2.404775      2.366824        0.281667   0.175864         3.043830   
 997     2.404775      2.689459        0.283947   0.175864         3.043830   
 998     2.404775      2.689459        0.281667   0.175864         3.043830   
 
      compactness_mean  concavity_mean  concave po

In [35]:
es_cf

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,-142.149034,-6.730307,-150.779048,-8315.515287,-7.063436,16.219015,-1.940906,-1.517383,175.932523,4.841332,...,-11.676076,-5.418281,-7.559553,-190.510342,-2.167585,-6.280738,-2.235629,-1.243863,-1.915985,-9.564705


In [36]:
cf

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
879,0.236703,0.296314,0.236151,0.128083,0.461812,0.327513,0.211372,0.217117,0.442,0.436443,...,0.206179,0.324355,0.195616,0.092334,0.481854,0.300679,0.226781,0.398017,0.399579,0.298769


In [37]:
np.array(cf - norm_t_data.iloc[2])

array([[-0.36479258, -0.09394609, -0.35959213, -0.32133411, -0.05249709,
        -0.1035033 , -0.25113968, -0.41856876, -0.06759637,  0.22519636,
        -0.14981183,  0.09258473, -0.10639956, -0.13357754,  0.05026251,
        -0.04963936, -0.01364475, -0.16537923,  0.09723029,  0.01121279,
        -0.35020657, -0.03571974, -0.31282565, -0.28217441, -0.0017356 ,
        -0.08469657, -0.13296303, -0.43703447, -0.00412684,  0.08533567]])

In [38]:
np.array(cf - norm_t_data.iloc[2])

array([[-0.36479258, -0.09394609, -0.35959213, -0.32133411, -0.05249709,
        -0.1035033 , -0.25113968, -0.41856876, -0.06759637,  0.22519636,
        -0.14981183,  0.09258473, -0.10639956, -0.13357754,  0.05026251,
        -0.04963936, -0.01364475, -0.16537923,  0.09723029,  0.01121279,
        -0.35020657, -0.03571974, -0.31282565, -0.28217441, -0.0017356 ,
        -0.08469657, -0.13296303, -0.43703447, -0.00412684,  0.08533567]])

In [39]:
clf.predict(es_cf)

array([1], dtype=int64)

In [40]:
from density_check import Kernel_obj
print(Kernel_obj(np.array(norm_t_data.loc[lab['labels'] == 1])).knn_density(np.array(cf)))

0.38135069305009456


In [41]:
from itertools import product

def cartesian_product_vectors(vectors):
    """
    Generate the Cartesian product of a set of vectors, assuming all vectors have the same dimension.

    Args:
    vectors (list of lists or tuples): List of vectors to form the Cartesian product.

    Returns:
    list of lists: A list of vectors with the same dimension formed by the Cartesian product.
    """
    cartesian_product = product(*vectors)
    return list(cartesian_product)

# Example usage:
vectors = [[1, 2, 3], ['A', 'B', 'C'], [10.0, 20.0, 30.0]]

ca = cartesian_product_vectors(vectors)
ca



[(1, 'A', 10.0),
 (1, 'A', 20.0),
 (1, 'A', 30.0),
 (1, 'B', 10.0),
 (1, 'B', 20.0),
 (1, 'B', 30.0),
 (1, 'C', 10.0),
 (1, 'C', 20.0),
 (1, 'C', 30.0),
 (2, 'A', 10.0),
 (2, 'A', 20.0),
 (2, 'A', 30.0),
 (2, 'B', 10.0),
 (2, 'B', 20.0),
 (2, 'B', 30.0),
 (2, 'C', 10.0),
 (2, 'C', 20.0),
 (2, 'C', 30.0),
 (3, 'A', 10.0),
 (3, 'A', 20.0),
 (3, 'A', 30.0),
 (3, 'B', 10.0),
 (3, 'B', 20.0),
 (3, 'B', 30.0),
 (3, 'C', 10.0),
 (3, 'C', 20.0),
 (3, 'C', 30.0)]

In [42]:
def cartesian_product_vectors(vectors):
    if not vectors:
        return [()]

    def recursive_cartesian_product(index):
        if index == len(vectors):
            return [()]
        current_vector = vectors[index]
        previous_product = recursive_cartesian_product(index + 1)
        result = []
        for value in current_vector:
            for product in previous_product:
                result.append([value] + list(product))
        return result

    return recursive_cartesian_product(0)

# Example usage:
vector1 = [1, 2]
vector2 = ['A', 'B']
vector3 = [10, 20]

vectors = [vector1, vector2, vector3]

result = cartesian_product_vectors(vectors)

for item in result:
    print(item)


[1, 'A', 10]
[1, 'A', 20]
[1, 'B', 10]
[1, 'B', 20]
[2, 'A', 10]
[2, 'A', 20]
[2, 'B', 10]
[2, 'B', 20]


In [43]:
import random
import numpy as np

def perturb_vector(vector, num_to_perturb, mean=0, std_dev=1):
    """
    Perturb a vector by replacing a specified number of its values with random samples
    from a Gaussian distribution.

    Args:
    - vector: The input vector (list) to be perturbed.
    - num_to_perturb: The number of values to perturb.
    - mean: The mean of the Gaussian distribution (default is 0).
    - std_dev: The standard deviation of the Gaussian distribution (default is 1).

    Returns:
    - The perturbed vector with some values replaced by Gaussian samples.
    """
    if num_to_perturb < 0 or num_to_perturb > len(vector):
        raise ValueError("num_to_perturb should be between 0 and the length of the vector")

    # Create a copy of the input vector to avoid modifying the original
    perturbed_vector = vector.copy()

    # Choose random indices to perturb
    indices_to_perturb = random.sample(range(len(perturbed_vector)), num_to_perturb)

    # Perturb the selected indices
    for index in indices_to_perturb:
        perturbed_vector[index] = np.random.normal(mean, std_dev)

    return perturbed_vector

# Example usage:
original_vector = [1.0, 2.0, 3.0, 4.0, 5.0]
num_to_perturb = 2
perturbed = perturb_vector(original_vector, num_to_perturb, mean=0, std_dev=0.1)
print(perturbed)


[-0.16959344442936902, 2.0, 3.0, 0.04144423469897254, 5.0]


In [44]:
import numpy as np
def synthetic_generator(unit, train_data, training_labels, target_class):
    target_points = train_data.loc[training_labels['labels'] == target_class]
    lst = []
    for i in range(len(unit)):
        for j in range(len(unit)):
            lst.append(perturb_vector(unit, i+1, mean=0, std_dev=0.1))
        

    

    return np.array(lst)

In [45]:
M = synthetic_generator(norm_t_data.iloc[0], norm_t_data, lab, 1)

In [46]:
def generate_subsets(nums):
    def backtrack(start, current_subset):
        # Add the current subset to the list of subsets
        subsets.append(current_subset[:])
        
        # Explore all possible options to form subsets
        for i in range(start, len(nums)):
            current_subset.append(nums[i])
            backtrack(i + 1, current_subset)
            current_subset.pop()

    subsets = []
    backtrack(0, [])
    return subsets

# Example usage:
input_set = [1, 2, 3]
all_subsets = generate_subsets(input_set)
for subset in all_subsets:
    print(subset)
all_subsets


[]
[1]
[1, 2]
[1, 2, 3]
[1, 3]
[2]
[2, 3]
[3]


[[], [1], [1, 2], [1, 2, 3], [1, 3], [2], [2, 3], [3]]

In [47]:
def l0_norm(vector):
    """
    Calculate the L0 norm of a vector, which is the count of non-zero elements in the vector.

    Args:
    - vector: The input vector (list or array).

    Returns:
    - The L0 norm of the vector (count of non-zero elements).
    """
    count = 0
    for element in vector:
        if element != 0:
            count += 1
    return count




In [48]:
import pandas as pd

data = {
    'Age': [25, 30, 22, 35, 28, 32, 19, 40, 27, 23],
    'Gender': ['Male', 'Female', 'Male', 'Female', 'Male', 'Female', 'Male', 'Male', 'Female', 'Male'],
    'Favorite Color': ['Blue', 'Red', 'Green', 'Blue', 'Red', 'Green', 'Blue', 'Green', 'Red', 'Blue'],
    'Salary': [50000.0, 60000.0, 45000.0, 75000.0, 55000.0, 68000.0, 42000.0, 80000.0, 63000.0, 47000.0],
    'Label': [0, 1, 0, 1, 0, 1, 0, 1, 0, 1]  # Example labels (0 and 1)
}

df = pd.DataFrame(data)

# Perform one-hot encoding for 'Gender' and 'Favorite Color'
df = pd.get_dummies(df, columns=['Gender', 'Favorite Color'])



In [49]:
labels = df['Label']
labe = pd.DataFrame(np.array(labels), columns=['labels'])
df.drop(['Label'], axis = 1, inplace = True)


In [50]:
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(df, labe, test_size=0.3, random_state=42)

In [51]:
clf.fit(X_train, y_train)

Iteration 1, loss = 15.44738673
Iteration 2, loss = 15.44737509
Iteration 3, loss = 15.44736601
Iteration 4, loss = 15.44735940
Iteration 5, loss = 15.44735470
Iteration 6, loss = 0.89220860
Iteration 7, loss = 0.90246690
Iteration 8, loss = 2.06987933
Iteration 9, loss = 0.91683860
Iteration 10, loss = 0.92161599
Iteration 11, loss = 0.92363420
Iteration 12, loss = 0.92334033
Iteration 13, loss = 15.44735540
Iteration 14, loss = 0.92167429
Iteration 15, loss = 0.92034532
Iteration 16, loss = 0.91740606
Iteration 17, loss = 0.91310328
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


C:\Users\amade\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1118: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPClassifier(hidden_layer_sizes=(6, 5), learning_rate_init=0.05,
              random_state=42, verbose=True)

In [52]:
clf.predict(df)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [53]:
new_explainer = CLEAR1(clf, 100,backend = 'sklearn', synthetic_generator_scheme='Gaussian',  classification_threshold = 0.90,cat_cols= ['Gender_Female', 'Gender_Male', 'Favorite Color_Blue', 'Favorite Color_Green', 'Favorite Color_Red'], number_of_CFEs = 200)

In [54]:
np.array(df.iloc[0])#.reshape((1, df.shape[1])), columns =df.columns)

array([2.5e+01, 5.0e+04, 0.0e+00, 1.0e+00, 1.0e+00, 0.0e+00, 0.0e+00])

In [55]:
new_explainer.fit(df, labe)

In [56]:
a, b = new_explainer.synthetic_generator(df, labe,df.iloc[0],1)

no numerical column specified


In [57]:
a

,Age,Salary,Gender_Female,Gender_Male,Favorite Color_Blue,Favorite Color_Green,Favorite Color_Red
0,25.000000,50000.000000,0,1,1,0,0
1,22.000000,45000.000000,0,1,0,1,0
2,28.000000,55000.000000,0,1,0,0,1
3,19.000000,42000.000000,0,1,1,0,0
4,27.000000,63000.000000,1,0,0,0,1
5,25.795889,57961.592110,0,1,0,1,0
6,21.341357,51156.098334,1,1,0,0,0
7,28.732336,39738.533515,0,1,0,0,0
8,26.675018,36239.918782,0,1,1,1,1
9,23.457041,47933.101472,0,1,0,0,1


In [58]:
clf.predict(a)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [59]:
new_explainer.generate_counterfactual(df.iloc[5])

no numerical column specified
no numerical column specified


C:\Users\amade\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 1

In [ ]:
labe

,labels
0,0
1,1
2,0
3,1
4,0
5,1
6,0
7,1
8,0
9,1
